## Summary notes

This is my solution to Advent of Code 2015, Day 5: *Some Assembly Required*

## Dependencies

In [1]:
import collections
import dataclasses
import laughingrook as lr

## Functions

In [2]:
@dataclasses.dataclass
class Wire:
    line: list[str]
    output: str = ''
    op: str = 'ASSIGN'
    inputs: list = dataclasses.field(default_factory=list)

    def __post_init__(self):
        self.parse_line()

    def parse_line(self) -> None:
        lline = self.line.split()
        self.output = lline[-1]

        left = lline[:-2]
        for op in ['NOT', 'AND', 'OR', 'LSHIFT', 'RSHIFT']:
            if op in left:
                self.op = op
                left.remove(op)
        self.inputs = [int(i) if i.isdigit() else i for i in left]

    def evaluate(self) -> None:
        if self.op == 'ASSIGN':
            return int(self.inputs[0])
        elif self.op == 'NOT':
            return int(65535 - self.inputs[0])
        elif self.op == 'AND':
            return int(self.inputs[0] & self.inputs[1])
        elif self.op == 'OR':
            return int(self.inputs[0] | self.inputs[1])
        elif self.op == 'LSHIFT':
            return int(self.inputs[0] << self.inputs[1])
        elif self.op == 'RSHIFT':
            return int(self.inputs[0] >> self.inputs[1])
        else:
            raise ValueError('invalid operator')

    def update_input(self, signals: dict) -> None:
        """Replace any signal in wire's inputs with int value.
        
        Iterate over wire's inputs: For signal in inputs, if signal
        in signals then add signals[signal] to inputs. Otherwise append
        signal to inputs
        """
        self.inputs = [signals[signal] if signal in signals else signal
                       for signal in self.inputs]

    def is_complete(self):
        """Return true if the wire's inputs are all of type int.
        """
        return all(isinstance(signal, int) for signal in self.inputs)

In [3]:
def evaluate(wires: list, override: tuple = None) -> dict:
    if not override:
        signals = {}
    else:
        signals = {override[0]: override[1]}
    q = collections.deque(wires)
    while len(q) >= 1:
        wire = q.popleft()
        if wire.is_complete():  # evaluate the string, add to signal map
            signals[wire.output] = wire.evaluate()
        else:                   # 
            wire.update_input(signals)
            q.append(wire)
    return signals

## Main

### Load the input

In [4]:
lines = lr.datasets.get_advent_input(2015, 7)
lines[:5]

file was cached.


['bn RSHIFT 2 -> bo',
 'lf RSHIFT 1 -> ly',
 'fo RSHIFT 3 -> fq',
 'cj OR cp -> cq',
 'fo OR fz -> ga']

Examples of an instance of `Wire`.

In [5]:
for w in [Wire(line) for line in lines[:3]]:
    print(w)

Wire(line='bn RSHIFT 2 -> bo', output='bo', op='RSHIFT', inputs=['bn', 2])
Wire(line='lf RSHIFT 1 -> ly', output='ly', op='RSHIFT', inputs=['lf', 1])
Wire(line='fo RSHIFT 3 -> fq', output='fq', op='RSHIFT', inputs=['fo', 3])


### Part 1

In [6]:
print(f"Solution = {evaluate([Wire(line) for line in lines])['a']}")

Solution = 46065


### Part 2

In [7]:
a = evaluate([Wire(line) for line in lines])['a']
wires = [wire for wire in [Wire(line) for line in lines]
         if wire.output != 'b']
print(f"Solution = {evaluate(wires, override=('b', a))['a']}")

Solution = 14134


### Performance

Part 1 =

In [8]:
%timeit {evaluate([Wire(line) for line in lines])['a']}

47.3 ms ± 229 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Part 2 =

In [9]:
%%timeit
a = evaluate([Wire(line) for line in lines])['a']
wires = [wire for wire in [Wire(line) for line in lines]
         if wire.output != 'b']
evaluate(wires, override=('b', a))['a']

94.8 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
